<a href="https://colab.research.google.com/github/jyotidabass/Model-json-and-weights-binary-preprocessed-tongue-coating-classifier/blob/main/Model_json_and_weights_binary_preprocessed_tongue_coating_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

In [ ]:
train = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)

train_dataset = train.flow_from_directory("/content/drive/MyDrive/TWO-CLASS COATED-UNCOATED/Train",
                                          target_size=(150,150),
                                          batch_size = 20,
                                          class_mode = 'binary')
                                         
test_dataset = test.flow_from_directory("/content/drive/MyDrive/TWO-CLASS COATED-UNCOATED/Train",
                                          target_size=(150,150),
                                          batch_size =20,
                                          class_mode = 'binary')

Found 467 images belonging to 2 classes.
Found 467 images belonging to 2 classes.


In [ ]:
test_dataset.class_indices

{'Coated tongue': 0, 'Non-coated tongue': 1}

In [ ]:
model = keras.Sequential()

# Convolutional layer and maxpool layer 1
model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 2
model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 3
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# Convolutional layer and maxpool layer 4
model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

# This layer flattens the resulting image array to 1D array
model.add(keras.layers.Flatten())

# Hidden layer with 512 neurons and Rectified Linear Unit activation function 
model.add(keras.layers.Dense(512,activation='relu'))

# Output layer with single neuron which gives 0 for Cat or 1 for Dog 
#Here we use sigmoid activation function which makes our model output to lie between 0 and 1
model.add(keras.layers.Dense(1,activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
#steps_per_epoch = train_imagesize/batch_size

model.fit_generator(train_dataset,
         steps_per_epoch = 10,
         epochs = 4,
         validation_data = test_dataset
       
         )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/4
10/10 [==============================] - 42s 4s/step - loss: 0.8070 - accuracy: 0.4600 - val_loss: 0.6934 - val_accuracy: 0.4989
Epoch 2/4
10/10 [==============================] - 30s 3s/step - loss: 0.6939 - accuracy: 0.4900 - val_loss: 0.6932 - val_accuracy: 0.5011
Epoch 3/4
10/10 [==============================] - 27s 3s/step - loss: 0.6934 - accuracy: 0.5027 - val_loss: 0.6930 - val_accuracy: 0.5011
Epoch 4/4
10/10 [==============================] - 27s 3s/step - loss: 0.6951 - accuracy: 0.5400 - val_loss: 0.6984 - val_accuracy: 0.5011


In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])

probability = model.predict(images, batch_size=10)
if probability > 0.5:
    print('Image is of a unCoated tongue.')
elif probability < 0.5:
    print('Image is of an coated tongue.')
else:
    print('Image is ambiguous. ')
os.remove(path)
img_array=tf.keras.utils.img_to_array(img)
img_array=tf.expand_dims(img_array,0)
predictions=model.predict(img_array)
score=tf.nn.softmax(predictions[0])

Saving 20220309_105103 - NITISH Kumar.jpg to 20220309_105103 - NITISH Kumar (1).jpg
Image is of a unCoated tongue.


In [ ]:
model_json = model.to_json()
with open("ImageClassifier.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("ImageClassifier.h5")

In [ ]:
def load_model(json_path, weight_path, metrics=None, loss=None, optimizer=None, custom_objects=None, is_compile=True):
    with open(json_path, 'r') as f:
        model_json_string = json.load(f)
    model_json_dict = json.loads(model_json_string)
    model = model_from_json(model_json_string, custom_objects=custom_objects)
    model.load_weights(weight_path)

    if is_compile:
        if optimizer is None:
            optimizer = model_json_dict['optimizer']['name']

        if loss is None:
            loss = model_json_dict['loss']

        if metrics is None:
            model.compile(loss=loss, optimizer=optimizer)
        else:
            model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

    return model 

In [ ]:
# some utilities
import os
import numpy as np

# Flask
from flask import Flask, redirect, url_for, request, render_template, Response, jsonify, redirect

#tensorflow
import tensorflow as tf
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2


# Variables 
# Change them if you are using custom model or pretrained model with saved weigths
Model_json = "/content/ImageClassifier.json"
Model_weigths = "/content/ImageClassifier.h5"


# Declare a flask app
app = Flask(__name__)

def get_ImageClassifierModel():
    model = MobileNetV2(weights='imagenet')

    # Loading the pretrained model
    # model_json = open(Model_json, 'r')
    # loaded_model_json = model_json.read()
    # model_json.close()
    # model = model_from_json(loaded_model_json)
    # model.load_weights(Model_weigths)

    return model  
    


def model_predict(img, model):
    '''
    Prediction Function for model.
    Arguments: 
        img: is address to image
        model : image classification model
    '''
    img = img.resize((224, 224))

    # Preprocessing the image
    x = image.img_to_array(img)
    # x = np.true_divide(x, 255)
    x = np.expand_dims(x, axis=0)

    # Be careful how your trained model deals with the input
    # otherwise, it won't make correct prediction!
    x = preprocess_input(x, mode='tf')

    preds = model.predict(x)
    return preds


@app.route('/', methods=['GET'])
def index():
    '''
    Render the main page
    '''
    return render_template('index.html')


@app.route('/predict', methods=['GET', 'POST'])
def predict():
    '''
    predict function to predict the image
    Api hits this function when someone clicks submit.
    '''
    if request.method == 'POST':
        # Get the image from post request
        img = base64_to_pil(request.json)
        
        # initialize model
        model = get_ImageClassifierModel()

        # Make prediction
        preds = model_predict(img, model)

        pred_proba = "{:.3f}".format(np.amax(preds))    # Max probability
        pred_class = decode_predictions(preds, top=1)   # ImageNet Decode

        result = str(pred_class[0][0][1])               # Convert to string
        result = result.replace('_', ' ').capitalize()
        
        # Serialize the result, you can add additional fields
        return jsonify(result=result, probability=pred_proba)
    return None


if __name__ == '__main__':
    # app.run(port=5002)
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
